Starting with Creating Random Data for 6 Students. and Saving it in a csv File

In [2]:
import csv
import random
data = []  
header = ["Student_ID","Subject","Marks"]
student_id = [1,2,3,4,5,6]
subject = ["p","c","m","b"]
data.append(header)
for std in student_id:
  for sub in subject:
    new_row=[std,sub,random.randint(0,100)]
    data.append(new_row)
            
with open("Random.csv", "w+") as Ran_file:
    writer = csv.writer(Ran_file, delimiter=",")
    for new_row in data:
        writer.writerow(new_row)

Installing required Java, Apache Spark, Hadoop

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

Setting Envionment Variables for Java and Spark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

Creating Local Session of Spark

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

dataset = spark.read.csv('Random.csv',inferSchema=True, header =True)

Showing Full data from csv file of all 6 students

In [17]:
dataset.show(25)

+----------+-------+-----+
|Student_ID|Subject|Marks|
+----------+-------+-----+
|         1|      p|    5|
|         1|      c|   35|
|         1|      m|   24|
|         1|      b|   54|
|         2|      p|   36|
|         2|      c|   28|
|         2|      m|   92|
|         2|      b|   95|
|         3|      p|   37|
|         3|      c|   34|
|         3|      m|   35|
|         3|      b|   11|
|         4|      p|   84|
|         4|      c|    8|
|         4|      m|   20|
|         4|      b|   45|
|         5|      p|   93|
|         5|      c|    2|
|         5|      m|   16|
|         5|      b|   22|
|         6|      p|    6|
|         6|      c|   80|
|         6|      m|   49|
|         6|      b|   75|
+----------+-------+-----+



Importing requiered liberaries for achieving our results

In [8]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col, avg, max, min, sum, row_number

**#1> Query to Rank students by total marks**

In [9]:
windowThree  = Window.partitionBy("Student_ID").orderBy('Subject')
dataset.withColumn("row",row_number().over(windowThree)) \
  .withColumn("Total", sum(col("Marks")).over(windowThree)) \
  .where(col("row")==4).select("Student_ID","Total") \
  .orderBy("Total", ascending=False) \
  .show()

+----------+-----+
|Student_ID|Total|
+----------+-----+
|         2|  251|
|         6|  210|
|         4|  157|
|         5|  133|
|         1|  118|
|         3|  117|
+----------+-----+



**#2> Query to find top three highest scorers for each subject**

In [10]:
window = Window.partitionBy(dataset['Subject']).orderBy(dataset['Marks'].desc())
dataset.select('*', rank().over(window).alias('Rank')).filter(col('Rank') <= 3).show() 

+----------+-------+-----+----+
|Student_ID|Subject|Marks|Rank|
+----------+-------+-----+----+
|         2|      m|   92|   1|
|         6|      m|   49|   2|
|         3|      m|   35|   3|
|         5|      p|   93|   1|
|         4|      p|   84|   2|
|         3|      p|   37|   3|
|         6|      c|   80|   1|
|         1|      c|   35|   2|
|         3|      c|   34|   3|
|         2|      b|   95|   1|
|         6|      b|   75|   2|
|         1|      b|   54|   3|
+----------+-------+-----+----+



**#3> Find min ,max and mean scores for each subjects**

In [11]:
windowThree  = Window.partitionBy("Subject").orderBy('Subject')
dataset.withColumn("row",row_number().over(windowThree)) \
  .withColumn("Mean", avg(col("Marks")).over(windowThree)) \
  .withColumn("Minimum", min(col("Marks")).over(windowThree)) \
  .withColumn("Maximum", max(col("Marks")).over(windowThree)) \
  .where(col("row")==1).select("Subject","Mean","Minimum","Maximum") \
  .show()

+-------+------------------+-------+-------+
|Subject|              Mean|Minimum|Maximum|
+-------+------------------+-------+-------+
|      m|39.333333333333336|     16|     92|
|      p|              43.5|      5|     93|
|      c|31.166666666666668|      2|     80|
|      b|50.333333333333336|     11|     95|
+-------+------------------+-------+-------+

